In [25]:
# Importing CSV files and display them as dataframes
import numpy as np
import pandas as pd
rating_dataset = pd.read_csv("ratings.csv")
rating_dataset = rating_dataset
rating_dataset = rating_dataset.drop(columns =['timestamp'])
rating_dataset.head()
movie_dataset = pd.read_csv("movies.csv")
movie_dataset = movie_dataset
#movie_dataset.head()

 
data = pd.merge(rating_dataset,movie_dataset, left_on='movieId',right_on ='movieId',how='left')

data.head()

,userId,movieId,rating,title,genres
0,1,2,3.5,Jumanji (1995),Adventure|Children|Fantasy
1,1,29,3.5,"City of Lost Children, The (Cité des enfants p...",Adventure|Drama|Fantasy|Mystery|Sci-Fi
2,1,32,3.5,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
3,1,47,3.5,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,3.5,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [26]:
# check if there is any data in any column missing or not available
data.isnull().values.any()

False

In [27]:
# find threshold by averging the movie ratings
# get only top5 rated movies per user,threshold is 4.5
averageRating = data.loc[data['rating'].idxmax()]['rating'] 
toprated_user_movies = data.loc[data['rating'] >= averageRating/2+1]
toprated_user_movies = toprated_user_movies.sort_values(['userId','rating'],ascending=[True,False])
toprated_user_movies.reset_index(inplace=True, drop=True)
toprated_user_movies.head()
# checking if user has rated a movie multiple times
#toprated_user_movies.duplicated(subset=['userId','movieId'])
# actual data is 20000263 after this step it is 646297

,userId,movieId,rating,title,genres
0,1,4993,5.0,"Lord of the Rings: The Fellowship of the Ring,...",Adventure|Fantasy
1,1,5952,5.0,"Lord of the Rings: The Two Towers, The (2002)",Adventure|Fantasy
2,1,7153,5.0,"Lord of the Rings: The Return of the King, The...",Action|Adventure|Drama|Fantasy
3,1,8507,5.0,Freaks (1932),Crime|Drama|Horror
4,1,1196,4.5,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Sci-Fi


In [28]:
# Break up the big genre string into a string array
toprated_user_movies['genres'] = toprated_user_movies['genres'].str.split('|')
# Convert genres to string value
toprated_user_movies['genres'] = np.array(toprated_user_movies['genres'])
toprated_user_movies.head()

,userId,movieId,rating,title,genres
0,1,4993,5.0,"Lord of the Rings: The Fellowship of the Ring,...","[Adventure, Fantasy]"
1,1,5952,5.0,"Lord of the Rings: The Two Towers, The (2002)","[Adventure, Fantasy]"
2,1,7153,5.0,"Lord of the Rings: The Return of the King, The...","[Action, Adventure, Drama, Fantasy]"
3,1,8507,5.0,Freaks (1932),"[Crime, Drama, Horror]"
4,1,1196,4.5,Star Wars: Episode V - The Empire Strikes Back...,"[Action, Adventure, Sci-Fi]"


In [36]:
#target domain
books = pd.read_csv('book_data.csv')
books = books[['book_title','genres','book_rating']]
# Break up the big genre string into a string array
books['genres'] = books['genres'].str.split('|')
# Convert genres to string value
books['genres'] = np.array(books['genres'])
books.head()

,book_title,genres,book_rating
0,The Hunger Games,"[Young Adult, Fiction, Science Fiction, Dystop...",4.33
1,Harry Potter and the Order of the Phoenix,"[Fantasy, Young Adult, Fiction]",4.48
2,To Kill a Mockingbird,"[Classics, Fiction, Historical, Historical Fic...",4.27
3,Pride and Prejudice,"[Classics, Fiction, Romance]",4.25
4,Twilight,"[Young Adult, Fantasy, Romance, Paranormal, Va...",3.58


In [37]:
def book_basedon_movie(genre):
    # finding intersection and recommending books
    for i,row in books.iterrows():
        movies_set = set(genre)
        books_set = set(row.genres)
        intersection = movies_set.intersection(books_set)
       
        score = float(len(intersection))/(len(movies_set)+len(books_set))
        print intersection, score
        if score*10>1:
            return row 

book = book_basedon_movie(toprated_user_movies['genres'][0])

print book

set(['Fantasy']) 0.142857142857
book_title                                      The Hunger Games
genres         [Young Adult, Fiction, Science Fiction, Dystop...
book_rating                                                 4.33
Name: 0, dtype: object
